In [ ]:
from mixer.models.modeling import MlpMixer, CONFIGS
import numpy as np

In [ ]:
mixer = MlpMixer(CONFIGS['Mixer-B_16'])
mixer.load_from(np.load("mixer/checkpoint/Mixer-B_16.npz"))
mixer

In [ ]:
from mixer.utils.data_utils import get_loader

In [ ]:
class Args:
    pass

args = Args()
args.local_rank = -1
args.img_size = 224
args.train_batch_size=8
args.eval_batch_size=8

In [ ]:
train_loader, test_loader = get_loader(args)

In [ ]:
#https://discuss.pytorch.org/t/how-can-i-extract-intermediate-layer-output-from-loaded-cnn-model/77301/3
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for i in range(0, 12):
    mixer.layer[i].register_forward_hook(get_activation(i))

# print(next(iter(train_loader)))
output = mixer(next(iter(train_loader))[0])
activation

In [23]:
print(activation[0].shape)
a = activation[0].mean(axis=0)
a.shape


torch.Size([8, 196, 768])


torch.Size([196, 768])

In [16]:
sigma = 1
type = 'kernel'

In [18]:
from CKA_Google import *

def Heatmap(result,type,sigma):
    dim = len(result)
    k = (dim - 1)
    heatmap_CKA = np.zeros((dim,dim))
    for i in range(0,dim):
        tr = (dim - 1)
        for j in range(0,dim):
            if type == 'kernel':
                heatmap_CKA[k][tr] = cka(gram_rbf(result[i],sigma),gram_rbf(result[j],sigma))
            elif type == 'linear':
                heatmap_CKA[k][tr] = cka(gram_linear(result[i]),gram_linear(result[j])) 
            else:
                print('There is no such category, try again')
                break

            tr -= 1
        k -= 1
    #print('CKA' + type + 'calculated')
    return heatmap_CKA

In [25]:
a.numpy().shape

(196, 768)

In [26]:
cka(gram_rbf(a.numpy(),sigma),gram_rbf(a.numpy(),sigma))

1.0